In [1]:
import os 
os.chdir("C:\\Users\\Microsoft\\AppData\\Local\\Programs\\Python\\Python310\\Scripts\\w5_img")

In [2]:
%pwd

'C:\\Users\\Microsoft\\AppData\\Local\\Programs\\Python\\Python310\\Scripts\\w5_img'

In [3]:
# setup 

import sys
# Python 3.7 is required
assert sys.version_info >= (3,7)

import cv2 as cv
import numpy as np

# For reproducibility,
np.random.seed(99)

# Make sure that optimization is enabled
if not cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

True

In [4]:
import matplotlib.pyplot as plt

# Q1

Convert the code chunk found under section Divide an image into smaller patches using cropping into a function with the following signature:
crop_grid(img, num_horizontal_grid, num_vertical_grid, line_color).
The output of the function should be image with grids.

In [5]:
def crop_grid(img, num_horizontal_grid, num_vertical_grid, line_color):
    src = cv.imread(img)
    src_copy = src.copy()
    
    height, width = src.shape[:2]
    num_hpatches = num_horizontal_grid 
    num_vpatches = num_vertical_grid 
    
    M, N = int(height/num_hpatches), int(width/num_vpatches)
    x1, y1 = 0, 0
    
    for y in range(0, height, M):
        for x in range(0, width, N):
            y1 = y + M 
            x1 = x + N
            
            if x1 >= width and y1 >= height:
                x1 = width - 1
                y1 = height - 1
                cv.rectangle(src_copy, (x, y), (x1, y1), line_color, 1)
                
            elif y1 >= height:
                y1 = height - 1
                cv.rectangle(src_copy, (x, y), (x1, y1), line_color, 1)
            
            elif x1 >= width:
                x1 = width - 1
                cv.rectangle(src_copy, (x, y), (x1, y1), line_color, 1)
        
            else:
                cv.rectangle(src_copy, (x, y), (x1, y1), line_color, 1)
                
    return src_copy


cv.imshow('patched image', crop_grid('dog.jfif', 3, 3, (0, 255, 255)))
cv.waitKey(0)
cv.destroyAllWindows()

# Q2

Display image sequences of smooth transition of two images with different values of α. Refer to code in section "Image blending". Use "lena.jfif" and "coins.jfif" as the base images.

In [6]:
img1 = cv.imread('lena.jfif')
img2 = cv.imread('coins.jfif')

# Resize img2
new_shape = img1.shape[:2]
img2 = cv.resize(img2, new_shape)

#define
for i in np.linspace(0,1,5):  #generate array with 5 num within the range
    alpha = i
    beta = 1 - alpha
    dst = cv.addWeighted(img1, alpha, img2, beta, 0)
    cv.imshow('Transition_img', dst)
    cv.waitKey(0)
    cv.destroyAllWindows()

# Q3

Rotate image by 45 degrees without cropping the sides of the image. (Hint: There are 2 strategies to tackle these problems). Use "lena.jfif" as the input image.

In [7]:
#First method: Use external libraries imutils

import imutils

img = cv.imread('lena.jfif')
rotated_img = imutils.rotate_bound(img, -45)
cv.imshow('rotatedimg', rotated_img)
cv.waitKey(0)
cv.destroyAllWindows()

In [8]:
#Second method: Modify the transformation matrix

img = cv.imread('lena.jfif')
h, w = img.shape[:2]  #cols:width, rows:height

M = cv.getRotationMatrix2D(((w-1)/2.0, (h-1)/2.0), 45, 1)

#take sin and cos value from M, use np absolute function for +ve value
cosofM = np.abs(M[0][0])
sinofM = np.abs(M[0][1])
  
#calculate new height and width of img to prevent cropping img sides 
newH = int((h * sinofM) + (w * cosofM))
newW = int((h * cosofM) + (w * sinofM))
  
#update the values of M
M[0][2] += (newW/2) - ((w-1)/2.0)
M[1][2] += (newH/2) - ((h-1)/2.0)
  

dst = cv.warpAffine(img, M, (newW, newH)) 

cv.imshow('rotated_image', dst)
cv.waitKey(0)
cv.destroyAllWindows()

# Q4

Use the images with titles: "flower.jfif" and "native-bee.png". I want to put flower above an image. If I add two images, it will change color. If I blend it, I get a transparent effect. But I want it to be opaque. If it was a rectangular region, we could use the ROI as we did in the previous section. But flower is not a rectangular region.This is where bitwise operations, like AND, OR, NOT and XOR really come in handy. The associated functions are cv.bitwise_and(), cv.bitwise_or() and cv.bitwise_not(). You need to use cv.threshold function to segment the flower

In [9]:
img1 = cv.imread('flower.jfif')
img2 = cv.imread('native-bee.png')
flower_grayscale = cv.cvtColor(img1,cv.COLOR_BGR2GRAY)

rows, columns, channels = img1.shape
roi = img2[0:rows, 0:columns] #determine roi

T, threshInv = cv.threshold(flower_grayscale, 100, 255, cv.THRESH_BINARY_INV) #threshold
img2_roi = cv.bitwise_and(roi, roi, mask=threshInv)
thresh_inv = cv.bitwise_not(threshInv)
flower = cv.bitwise_and(img1, img1, mask=thresh_inv)

flower_bee = cv.add(flower, img2_roi) #combine
img2[0:rows, 0:columns] = flower_bee  #replace into img2

cv.imshow('flower_bee', img2)
cv.waitKey(0)
cv.destroyAllWindows()